In [2]:
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
import os
import pickle

# If modifying these SCOPES, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

def main():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "C:/Users/punit/Downloads/client_secret_10654816956-2fv0p2pllm58u8ie8ivgr0h2rep17tbv.apps.googleusercontent.com.json", SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)

    # Call the Gmail API
    results = service.users().messages().list(userId='me',maxResults=10).execute()
    messages = results.get('messages', [])

    if not messages:
        print('No new messages.')
    else:
        print('Message IDs:')
        for message in messages:
            print(message['id'])

if __name__ == '__main__':
    main()


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=10654816956-2fv0p2pllm58u8ie8ivgr0h2rep17tbv.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A52386%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=FFS3MB8CMrjSRI8T6eGt8oVWDKOTIH&access_type=offline
Message IDs:
18964fc213d939dd
18964fc02a91b235
18964fbed30f3075
18964fbdab5598a5
18964fbc9ac9109f
18964fbafc4f21d6
18964fb9a4a9b7a1
18964fb85bcf2307
18964fb6e82ca724
18964fb5579be516


In [1]:
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from base64 import urlsafe_b64decode
from email.mime.text import MIMEText
import os
import time
import dateutil.parser as parser
from datetime import datetime
import datetime
import pandas as pd
import pickle
import re
import nltk

# If modifying these SCOPES, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

def clean_email_body(body):
    # split the email body into sections based on a pattern of lines that are completely blank
    sections = re.split(r'\n\s*\n', body)

    # define the keywords that are likely to appear in a signature or disclaimer
    signature_disclaimer_keywords = [
        'Regards', 'Sincerely', 'Best', 'Thank', 'Thank you', 'Cheers',
        'confidential', 'intended for', 'may not', 'without the written consent', 
        'phone:', 'email:', 'www', 'http', '.com', 'from:', 'sent:', 'subject:', 'to:',
    ]

    # filter out sections that are likely to be part of a signature, disclaimer, or header
    sections = [section for section in sections if not (
        len(section) < 50 or  # exclude short sections
        len(nltk.sent_tokenize(section)) < 2 or  # exclude sections with less than 2 sentences
        any(keyword.lower() in section.lower() for keyword in signature_disclaimer_keywords)  # exclude sections with certain keywords
    )]

    # join the remaining sections back into a single string
    cleaned_body = '\n\n'.join(sections)

    return cleaned_body


def parse_parts(parts):
    """
    Utility function that parses the body of the message parts
    """
    data = ""
    if parts:
        for part in parts:
            mimeType = part.get('mimeType')
            body = part.get('body')
            data = body.get('data')
            
            if part.get('parts'):
                # recursively call this function when we see that a part
                # has parts inside
                return parse_parts(part.get('parts'))
            
            if mimeType == "text/plain":
                # if the email part is text/plain type, we decode it
                text = urlsafe_b64decode(data).decode()
                return text

def extract_forwarded_message(body):
    # Assuming the forwarded message is indicated by "-----Original Message-----" or similar.
    # Please update this according to the format of your forwarded messages.
    pattern = r"(?<=-----Original Message-----)([\s\S]*)"
    match = re.search(pattern, body)
    return match.group(1) if match else None

def main():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "C:/Users/punit/Downloads/client_secret_10654816956-2fv0p2pllm58u8ie8ivgr0h2rep17tbv.apps.googleusercontent.com.json", SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    try:
        service = build('gmail', 'v1', credentials=creds)
        results = service.users().messages().list(userId='me').execute()
        messages = results.get('messages', [])
        
        message_count = 0
        df = pd.DataFrame(columns=['Date-Time', 'Sender', 'Subject', 'Body', 'Forwarded Message'])
        for msg in messages:
            txt = service.users().messages().get(userId='me', id=msg['id']).execute()
            try:
                payload = txt['payload']
                headers = payload['headers']

                for d in headers:
                    if d['name'] == 'Date':
                        date_time = parser.parse(d['value'])
                        date_time = date_time.strftime('%d-%b-%Y %H:%M:%S')    
                    if d['name'] == 'From':
                        sender = d['value']
                    if d['name'] == 'Subject':
                        subject = d['value']

                body = parse_parts(payload.get('parts'))
                cleaned_body = clean_email_body(body)
                forwarded_message = extract_forwarded_message(body)
                
                df = df.append({'Date-Time': date_time, 'Sender': sender, 'Subject': subject, 'Body': cleaned_body, 'Forwarded Message': forwarded_message}, ignore_index=True)
                message_count += 1
            except HttpError as error:
                print(f'An error occurred: {error}')

        print(f'Process completed. {message_count} messages fetched.')
        df.to_csv('emails.csv', index=False)
        print('Data saved to emails.csv')

        return df

    except Exception as error:
        print(f'An error occurred: {error}')
        return None

# if __name__ == '__main__':
#     main()



In [10]:
df

,Date-Time,Sender,Subject,Body,Forwarded Message
0,17-Jul-2023 17:53:10,Puneet Mishra <puneet.mishra@hertshtengroup.com>,Fw: PVM Fundamental Report April 26 2023,Hopes of a nascent price recovery in the oil m...,None
1,17-Jul-2023 17:53:02,Puneet Mishra <puneet.mishra@hertshtengroup.com>,Fw: PVM Fundamental Report April 25 2023,OPEC has been vindicated and its critics have ...,None
2,17-Jul-2023 17:52:57,Puneet Mishra <puneet.mishra@hertshtengroup.com>,Fw: PVM Fundamental Report April 24 2023,Friday’s modest rebound couldn’t prevent the c...,None
3,17-Jul-2023 17:52:52,Puneet Mishra <puneet.mishra@hertshtengroup.com>,Fw: PVM Fundamental Report April 21 2023,Oil markets were a sea of red yesterday. Brent...,None
4,17-Jul-2023 17:52:47,Puneet Mishra <puneet.mishra@hertshtengroup.com>,Fw: PVM Fundamental Report April 20 2023,"After four straight weekly advances, the oil c...",None
5,17-Jul-2023 17:52:41,Puneet Mishra <puneet.mishra@hertshtengroup.com>,Fw: PVM Fundamental Report April 19 2023,Upbeat data out of China usually provides a bo...,None
6,17-Jul-2023 17:52:35,Puneet Mishra <puneet.mishra@hertshtengroup.com>,Fw: PVM Fundamental Report April 18 2023,Oil bulls were in retreat mode yesterday as a ...,None
7,17-Jul-2023 17:52:30,Puneet Mishra <puneet.mishra@hertshtengroup.com>,Fw: PVM Fundamental Report April 17 2023,The markets are in limbo and understandably so...,None
8,17-Jul-2023 17:52:25,Puneet Mishra <puneet.mishra@hertshtengroup.com>,Fw: PVM Fundamental Report April 14 2023,Recessionary alarmism is back in the headlines...,None
9,17-Jul-2023 17:52:18,Puneet Mishra <puneet.mishra@hertshtengroup.com>,Fw: PVM Fundamental Report April 13 2023,There is a smorgasbord of reasons to be scepti...,None


In [2]:
df = main()

Process completed. 17 messages fetched.
Data saved to emails.csv


In [20]:
df['Body'].iloc[0]

'Hopes of a nascent price recovery in the oil markets went up in smoke yesterday. Brent and WTI snapped a two-day run of gains as US banking jitters resurfaced. Shares in First Republic plunged by 50% and in doing so reignited fears over the health of the banking sector. The sharp drop came a day after the embattled US regional bank revealed that customers had withdrawn $100 billion during the first-quarter crisis. Yesterday’s rout on Wall Street suggests that the banking crisis may still have another sting in its tail.\r\n\nAlongside renewed banking fears, risk appetite was undermined by mixed earnings reports and yet more downbeat US macro data. The Conference Board’s survey of consumer confidence fell to a nine-month low in April. This will add credence to claims that the US economy is edging closer to a recession. Meanwhile, investors’ negative mindset concerning the health of the world’s biggest economy lent support to the safe-haven dollar. The US currency added 0.5% against a ba

In [4]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from gensim.summarization import summarize

nltk.download('vader_lexicon')

def analyze_and_summarize(text):
    # initialize VADER
    sid = SentimentIntensityAnalyzer()

    # get sentiment scores
    sentiment_scores = sid.polarity_scores(text)
    sentiment = 'positive' if sentiment_scores['compound'] > 0 else 'negative'
    print(f'Sentiment: {sentiment}')

    # get summary
    summary = summarize(text, ratio=0.1)  # adjust the ratio parameter to control the length of the summary
    print(f'Summary:\n{summary}')

text = df['Body'].iloc[0]
analyze_and_summarize(text)

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [21]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

# Load pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

# Assume "context" is the text of the emails and "question" is what you're asking
context = "Oil markets were a sea of red yesterday. Brent and WTI shed around $2/bbl with the former coming perilously close to losing the $80/bbl handle. At the core of the current bout of price malaise are concerns that rising interest could hit economic growth. The Fed, BoE and ECB are all expected to hike rates next month as they continue to battle stubbornly high inflation. In the meantime, downbeat US economic data released on Thursday stoked fears of a looming recession in the world’s biggest economy. The number of Americans filing new claims for unemployment support rose last week by slightly more than expected. Shortly after, figures revealed that existing home sales fell in March as rising borrowing costs kept buyers on the sidelines. Meanwhile, a survey from the Philadelphia Fed Reserve showed its measure of factory activity in the mid-Atlantic region plunged to the lowest level in nearly three years in April. Economic and interest rate uncertainty is front-and-centre of the oil market’s focus. Nevertheless, near and medium-term oil fundamentals remain supportive. Global oil stocks are forecast to begin drawing from next month on the back of fresh OPEC cuts and accelerating Chinese demand. In view of this, the oil market should soon find a bottom and undergo an upward bounce."
question = "What is the main concern for oil markets?"

encoding = tokenizer.encode_plus(
  question, context,
  max_length=512,
  padding='max_length',
  truncation=True,
  return_attention_mask=True,
)

input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

# # Apply the tokenizer to the input text, treating them as a text-pair.
# input_ids = tokenizer.encode(question, context)

# # Get the input sequence's corresponding attention mask
# attention_mask = tokenizer.get_special_tokens_mask(input_ids)

# Run the model
outputs = model(torch.tensor([input_ids]), attention_mask=torch.tensor([attention_mask]))

# The start and end tokens with the highest scores
answer_start = torch.argmax(outputs.start_logits)
answer_end = torch.argmax(outputs.end_logits)

# Get the string versions of the input tokens.
tokens = tokenizer.convert_ids_to_tokens(input_ids)

# Start with the first token.
answer = tokens[answer_start]

# Select the remaining answer tokens and join them with whitespace.
for i in range(answer_start + 1, answer_end + 1):
    
    # If it's a subword token, then recombine it with the previous token.
    if tokens[i][0:2] == '##':
        answer += tokens[i][2:]
    
    # Otherwise, add a space then the token.
    else:
        answer += ' ' + tokens[i]

print('Answer: "' + answer + '"')



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_out

Answer: "on thursday stoked fears of a looming recession in the world ’ s biggest economy . the number of americans filing new claims for unemployment support rose last week by slightly more than expected . shortly after , figures revealed that existing home sales fell in march as rising borrowing costs kept buyers on the sidelines . meanwhile , a survey from the philadelphia fed reserve showed its measure of factory activity in the mid - atlantic region plunged to the lowest level in nearly three years in april . economic and interest rate uncertainty is front - and - centre of the oil market ’ s focus . nevertheless , near and medium - term oil fundamentals remain supportive . global oil stocks are forecast to begin drawing from next month on the back of fresh opec cuts and accelerating chinese demand . in view of this , the oil market should soon find a bottom and undergo an upward bounce . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

In [5]:
summary

['Brent and WTI snapped a two-day run of gains as US banking jitters resurfaced. Shares in First Republic plunged by 50% and reignited fears over the health of the banking sector. The Conference Board’s survey of consumer confidence fell to a nine-month low in April.']

In [7]:
summary

['OPEC has been vindicated and its critics have been left red-faced on the past two occasions – at least for now. Oil retreated from the peak reached a fortnight ago. The widely anticipated gasoil and distillate shortage that was supposed to take place after the introduction of the G7 embargo on February 5 has never materialized.']

In [9]:
summary

['Brent and WTI posted weekly falls of more than 5% and in doing so reversed all their gains from the OPEC cuts. Concerns that the global economy is slowing down and that oil demand will consequently take a hit are at the forefront of investors’ minds. Rising interest rates are not conducive to economic growth nor improving oil demand.']